### LLMs

LLMs implement the Runnable interface, the basic building block of the LangChain Expression Language (LCEL). This means they support `invoke`, `ainvoke`, `stream`, `astream`, `batch`, `abatch`, `astream_log` calls.

LLMs accept strings as inputs, or objects which can be coerced to string prompts, including List[BaseMessage] and PromptValue.

In [2]:
from dotenv import load_dotenv
load_dotenv()

from langchain.llms import OpenAI
llm = OpenAI(temperature=0)

## invoke
res = llm.invoke("What are some theories about the relationship between unemployment and inflation?")
print(res)
print("-----------------")

## stream
res = llm.stream("What are some theories about the relationship between unemployment and inflation?")
# type(res) = Generator
for chunk in res:
    print(chunk, end="", flush=True)
print("-----------------")    



1. The Phillips Curve: This theory suggests that there is an inverse relationship between unemployment and inflation. As unemployment decreases, inflation increases, and vice versa.

2. The NAIRU Theory: This theory suggests that there is a non-accelerating inflation rate of unemployment (NAIRU). This means that when unemployment is below the NAIRU, inflation will increase, and when unemployment is above the NAIRU, inflation will decrease.

3. The Cost-Push Theory: This theory suggests that when the cost of production increases, it leads to higher prices and inflation. This can be caused by an increase in wages due to high unemployment, which can lead to higher prices and inflation.

4. The Demand-Pull Theory: This theory suggests that when there is an increase in demand for goods and services, it leads to higher prices and inflation. This can be caused by an increase in consumer spending due to low unemployment, which can lead to higher prices and inflation.
-----------------


1. T

In [5]:
## batch
res = llm.batch(["What are some theories about the relationship between unemployment and inflation?"])
print(res)

## ainvoke
await llm.ainvoke("What are some theories about the relationship between unemployment and inflation?")

## astream
async for chunk in llm.astream("What are some theories about the relationship between unemployment and inflation?"):
    print(chunk, end="", flush=True)
    
## abatch
await llm.abatch(["What are some theories about the relationship between unemployment and inflation?"])

['\n\n1. The Phillips Curve: This theory suggests that there is an inverse relationship between unemployment and inflation. As unemployment decreases, inflation increases, and vice versa.\n\n2. The NAIRU Theory: This theory suggests that there is a non-accelerating inflation rate of unemployment (NAIRU). This means that when unemployment is below the NAIRU, inflation will increase, and when unemployment is above the NAIRU, inflation will decrease.\n\n3. The Cost-Push Theory: This theory suggests that when the cost of production increases, it leads to higher prices and inflation. This can be caused by an increase in wages due to high unemployment, which can lead to higher prices and inflation.\n\n4. The Demand-Pull Theory: This theory suggests that when there is an increase in demand for goods and services, it leads to higher prices and inflation. This can be caused by an increase in consumer spending due to low unemployment, which can lead to higher prices and inflation.']


1. The Phi

In [7]:
## astram_log
# async for chunk in llm.astream_log("What are some theories about the relationship between unemployment and inflation?"):
#     print(chunk)

In [32]:
## Simplest way to use LLM is a callable:
# pass in a string, get a string completion

## __call__
llm("Tell me joke.")

## generate
# response can include things like multiple top responses and other LLM provider-specific information:
llm_result = llm.generate(["Tell me a joke", "Tell me a poem"])
print(len(llm_result.generations))
# langchain.schema.output.LLMResult
print(llm_result)

# LLMResult has parameter llm_output - This information is not standardized across providers.
print(llm_result.llm_output)

2
generations=[[Generation(text='\n\nQ: What did the fish say when it hit the wall?\nA: Dam!', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nRoses are red,\nViolets are blue,\nSugar is sweet,\nAnd so are you.', generation_info={'finish_reason': 'stop', 'logprobs': None})]] llm_output={'token_usage': {'prompt_tokens': 8, 'completion_tokens': 46, 'total_tokens': 54}, 'model_name': 'text-davinci-003'} run=[RunInfo(run_id=UUID('f4ecfa64-014d-4814-9d3a-3b8553d9bab6')), RunInfo(run_id=UUID('11ee152a-daec-49b4-8cfd-8c3e24ff55c9'))]
{'token_usage': {'prompt_tokens': 8, 'completion_tokens': 46, 'total_tokens': 54}, 'model_name': 'text-davinci-003'}


In [34]:
import time
import asyncio
from langchain.llms import OpenAI

# async using agenerate method
async def async_generate(llm):
    res = await llm.agenerate(["Tell me a joke."])
    print(res.generations[0][0].text)

async def generate_concurrently():
    llm = OpenAI(temperature=.9)
    tasks = [async_generate(llm) for _ in range(5)]
    await asyncio.gather(*tasks)

await generate_concurrently()



Q: What did the fish say when it hit the wall?
A: Dam!


Q: What did the fish say when he hit the wall?
A: Dam!


Q: What did the fish say when it hit the wall?
A: Dam!


Q: Why don't scientists trust atoms? 
A: Because they make up everything!


Q: Why don't scientists trust atoms? 
A: Because they make up everything!


In [36]:
## Custom LLM
# _call method: required
from typing import Any, List, Mapping, Optional
from langchain.llms.base import LLM
from langchain.callbacks.manager import CallbackManagerForLLMRun

class CustomLLM(LLM):
    n: int

    @property
    def _llm_type(self) -> str:
        return "custom"

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")
        return prompt[: self.n]

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {"n": self.n}
    
llm = CustomLLM(n=10)
print(llm("This is a foobar thing"))
print(llm)

This is a 
CustomLLM
Params: {'n': 10}
